In [1]:
"""
pickle to dataframe
"""

'\npickle to dataframe\n'

In [ ]:
# imports

import re
import pickle
import os

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
news_files=os.listdir('news_crawling')
print(len(set(sorted([i.split("_")[-1].split('.')[0] for i in news_files]))))

In [ ]:
news_brands = list({i.split('_')[0] for i in news_files})

In [ ]:
# read_pickles by news_brands
for news_brand in news_brands:
    pickle_files = [i for i in news_files if i.startswith(news_brand)]
    pickle_dates=[i.split('_')[-1].split('.')[0] for i in pickle_files] 
    vars()[news_brand] = dict()
    for pickle_date,pickle_file in zip(pickle_dates,pickle_files):
        with open(f'news_crawling/{pickle_file}', 'rb') as f:
            vars()[news_brand][pickle_date] = pickle.load(f)

In [ ]:
# save pickles by news_brands
for news_brand in news_brands:
    with open(f"preprocessed_1_/{news_brand}.pickle","wb") as f:
        pickle.dump(vars()[news_brand],f)

In [ ]:
# dataframe based on news_brands
df_list = []
for news_brand in news_brands:
    df_list.append(
        pd.DataFrame.from_dict(vars()[news_brand],
                               orient='index',
                               columns=[f'{news_brand}_{i}' for i in range(1,6)])
    )

In [ ]:
# concat datframe
df_tot=pd.concat(df_list,axis=1).sort_index()